# Mancala: Player vs AI
In this project we have implemented a playable version of mancala using python with **slight rule tweaks**. We then proceeded to go try and test a few different types of AI to use as an opposing "computer" player for the humans to play against. These strategies were tested in this order:
1. Random moves.
2. Minimax
3. Minimax with alpha beta pruning

Our goal is to produce an AI that plays signficantly better than the random AI while running in real time without the user or player having to wait for long periods of time while the AI computes.
## Rule changes to our mancala
Our game of Mancala plays much like the normal version of mancala. We have varying sized boards that don't impact gameplay, but our main tweak to the original Mancala is:
1. If your last stone for your turn lands in your Mancala, you don't get to make another move. In other words, the turn always alternates every time a player makes a move without exception.
## How to play
In the cell below we define a series of variables with comments explaining what they do. You can simply toggle on your preferred variables and then run the code. If a player is playing, you will have to send your moves through the CLI by typing a valid number seen in the printed Mancala board.

In [ ]:
PLAY_ORIGINAL_GAME = False # temp variable, whether or not to play the basic mancala we implemented in hw7. Useful to avoid the "input" prompt coming up everytime we run the code
MOVE_COUNT = 10 # number of moves (both players combined) before the game ends automatically

PLAY_NEW_VERSION = False # temp variable, whether or not to play the new version using subclass of Game
PLAY_MINIMAX_VERSION = False # temp variable, whether or not to play the minimax version using subclass of Game
PLAY_ALPHABETA_VERSION = True # temp variable, whether or not to play the alphabeta version using subclass of Game

# Libraries used
Our libraries used are imported below. Most importantly, we are using the AIMA library game abstract class such that we will also have access to their minimax and alpha beta pruning algorithms.

In [ ]:
import random
from random import seed
from random import randint
random.seed(109)

In [ ]:
# from games4e import *
from utils4e import *

# Basic interactive Mancala game vs random player
Below is a simple implementation of Mancala with no superclass. It defines a variable-sized 2-player game using the rule exceptions we defined above. It uses a random move algorithm as the AI player.

In [ ]:
class Mancala:
    def __init__(self, pits_per_player, stones_per_pit):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        # write your code here
        if(self.current_player == 1):
            if (pit > 0 and pit < (self.pits_per_player + 1)) and (self.board[pit-1] > 0): 
                return True
            else:
                return False
        elif(self.current_player == 2):
            if (pit > 0 and pit < (self.pits_per_player + 1)) and (self.board[self.pits_per_player + pit] > 0): 
                return True
            else:
                return False
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        # write your code here
        value = randint(1, self.pits_per_player)
        while(self.board[value + self.pits_per_player] == 0):
            value = randint(1, self.pits_per_player)
        return value
    
    def play(self, pit):
            """
            This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
            1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
            2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
            3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

            Finally, the function then switches the current player, allowing the other player to take their turn.
            """
            print("Player", self.current_player, "chooses pit:", pit)
            # 1. Check valid move
            if not self.valid_move(pit):
                print("INVALID MOVE")
                return
            
            # DISTRIBUTE STONES
            if self.current_player == 1:
                start = self.p1_pits_index[0] + pit - 1
            else:
                start = self.p2_pits_index[0] + pit - 1

            stones = self.board[start]
            self.board[start] = 0

            while stones > 0:
                start = (start + 1) % len(self.board)  # Move to the next pit, wrapping around
                if self.current_player == 1 and start == self.p2_mancala_index:
                    continue  # Skip opponent's Mancala
                if self.current_player == 2 and start == self.p1_mancala_index:
                    continue  # Skip opponent's Mancala
                
                self.board[start] += 1
                stones -= 1

            # Capture stones if the last stone lands in an empty pit on the player's side
            if self.current_player == 1 and self.p1_pits_index[0] <= start <= self.p1_pits_index[1]:
                if self.board[start] == 1:  # Empty pit
                    opposite_pit = self.p2_pits_index[1] - (start - self.p1_pits_index[0])
                    self.board[self.p1_mancala_index] += self.board[start] + self.board[opposite_pit]
                    self.board[start] = 0
                    self.board[opposite_pit] = 0
            elif self.current_player == 2 and self.p2_pits_index[0] <= start <= self.p2_pits_index[1]:
                if self.board[start] == 1:  # Empty pit
                    opposite_pit = self.p1_pits_index[1] - (start - self.p2_pits_index[0])
                    self.board[self.p2_mancala_index] += self.board[start] + self.board[opposite_pit]
                    self.board[start] = 0
                    self.board[opposite_pit] = 0

            self.moves.append((self.current_player, pit))
            
            # 3. Switch player
            self.current_player = 2 if self.current_player == 1 else 1
            
            # 2. Winning evaluation, check for winner
            if self.winning_eval():
                print("GAME OVER")
                return

            return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_empty = self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1] == [0] * self.pits_per_player
        p2_empty = self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1] == [0] * self.pits_per_player
        return p1_empty or p2_empty

In [ ]:
import sys
game = Mancala(6, 4)
movecount = MOVE_COUNT # num of moves to allow in total
game.display_board()

# Loop to play game via user input if PLAY_GAME is True
if PLAY_ORIGINAL_GAME:
    for i in range(0, movecount):
        pit = None
        if game.winning_eval():
            print("Game ended, no more valid moves for Player", game.current_player)
            break

        if game.current_player == 1:
            try:
                pit = int(input("Player 1, choose a pit: ").strip())
                print("Input received:", pit)
            except:
                print("Invalid input. Try again")
            if not pit or not game.valid_move(pit):
                print("Invalid move \"" + str(pit) + "\". Try again")
                continue
        else:
            pit = game.random_move_generator()
        if pit:
            game.play(pit)
            game.display_board()
            print()
        else:
            print("No valid moves for Player", game.current_player)
            break

    # Game ended, print information
    print("\nList of valid moves:")
    for move in game.moves:
        player, pit = move
        print(f"Player {player} selected pit {pit}")


# Subclassing AIMA Game for Mancala
Below is where we leverage the AIMA library. By subclassing their game class with proper methods, we can then apply minimax and alpha beta pruning to our AI player to imrpove it's effectiveness.

In [ ]:
class Game:
    """A game is similar to a problem, but it has a utility for each
    state and a terminal test instead of a path cost and a goal
    test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.current_player

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    # return self.utility(state, self.to_move(self.initial))
                    return state

In [ ]:
# GameState contains the current state of the game
from collections import namedtuple
import random
import sys

# Update GameState to use current_player instead of to_move
GameState = namedtuple('GameState', ['current_player', 'utility', 'board', 'score'])

class Mancala(Game):
    """
    pits_per_player: Number of pits for each player
    stones_per_pit: Number of stones in each pit at the start of the game
    board_size: Total number of pits on the board
    initial: Initial state of the game
    p1_store: Index of player 1's store
    p2_store: Index of player 2's store
    p1_pits_range: Range of indices for player 1's pits
    p2_pits_range: Range of indices for player 2's pits
    """ 
    def __init__(self, pits_per_player=3, stones_per_pit=2):
        self.pits_per_player = pits_per_player
        self.stones_per_pit = stones_per_pit
        
        # Calculate total board size: pits for both players + 2 stores
        self.board_size = 2 * pits_per_player + 2
        
        # Initialize board with specified stones in each pit
        board = [stones_per_pit] * self.board_size
        board[pits_per_player] = 0  # Player 1's store
        board[-1] = 0  # Player 2's store
        
        # Initialize with player 1
        self.initial = GameState(current_player='1', utility=0, 
                               board=board, score={'1': 0, '2': 0})
        
        # Store indices for easy reference
        self.p1_store = pits_per_player
        self.p2_store = self.board_size - 1
        
        # Store pit ranges for display
        self.p1_pits_range = (0, pits_per_player - 1)
        self.p2_pits_range = (pits_per_player + 1, self.board_size - 2)

    def actions(self, state):
        """Legal moves are the indices of non-empty pits on the current player's side."""
        if state.current_player == '1':
            return [i for i in range(self.pits_per_player) if state.board[i] > 0]
        else:
            return [i for i in range(self.p1_store + 1, self.p2_store) 
                   if state.board[i] > 0]

    def result(self, state, move):
        """Return the state that results from making a move."""
        if move not in self.actions(state):
            return state  # Illegal move has no effect
            
        board = state.board.copy()
        score = state.score.copy()
        current_player = state.current_player
        
        # Pick up stones
        stones = board[move]
        board[move] = 0
        
        # Distribute stones
        current_pit = move
        
        while stones > 0:
            current_pit = (current_pit + 1) % self.board_size
            # Skip opponent's store
            if (current_player == '1' and current_pit == self.p2_store) or \
               (current_player == '2' and current_pit == self.p1_store):
                continue
                
            board[current_pit] += 1
            stones -= 1
            
            # Check for capture on last stone
            if stones == 0:
                if board[current_pit] == 1:
                    # Check if last stone landed in an empty pit on player's side
                    is_p1_side = current_pit < self.p1_store
                    is_p2_side = self.p1_store < current_pit < self.p2_store
                    
                    if (current_player == '1' and is_p1_side) or \
                       (current_player == '2' and is_p2_side):
                        opposite = (2 * self.pits_per_player) - current_pit
                        if board[opposite] > 0:
                            store = self.p1_store if current_player == '1' else self.p2_store
                            board[store] += board[opposite] + 1
                            board[opposite] = 0
                            board[current_pit] = 0
        
        # Update scores
        score['1'] = board[self.p1_store]
        score['2'] = board[self.p2_store]
        
        # Switch to other player
        next_player = '2' if current_player == '1' else '1'
        
        return GameState(current_player=next_player,
                        utility=self.compute_utility(board, score),
                        board=board,
                        score=score)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 for ongoing."""
        return state.utility if player == '1' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if one player's pits are all empty."""
        player1_empty = all(state.board[i] == 0 
                          for i in range(self.pits_per_player))
        player2_empty = all(state.board[i] == 0 
                          for i in range(self.p1_store + 1, self.p2_store))
        
        next_player = '2' if state.current_player == '1' else '1'
        return player1_empty or player2_empty

    def display(self, state):
        """Display the current state of the board in a user-friendly format."""
        board = state.board
        
        # Extract relevant sections of the board
        p1_pits = board[self.p1_pits_range[0]:self.p1_pits_range[1] + 1]
        p1_mancala = board[self.p1_store]
        p2_pits = board[self.p2_pits_range[0]:self.p2_pits_range[1] + 1]
        p2_mancala = board[self.p2_store]

        # Display board
        print('\nP1               P2')
        print('     ____{}____     '.format(p2_mancala))
        
        for i in range(self.pits_per_player):
            pit_num_1 = i  # Use actual index for player 1
            pit_num_2 = self.p2_pits_range[0] + (self.pits_per_player - 1 - i)  # Calculate actual index for player 2
            
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(
                    pit_num_1, p1_pits[i], p2_pits[-(i+1)], pit_num_2))
            else:    
                print('{} -> | {} | {} | <- {}'.format(
                    pit_num_1, p1_pits[i], p2_pits[-(i+1)], pit_num_2))
            
        print('         {}         '.format(p1_mancala))
        print('Turn: P{}'.format(state.current_player))
        print(f"Scores - P1: {state.score['1']}, P2: {state.score['2']}\n")

    def compute_utility(self, board, score):
        """Compute the utility value of the current state."""
        # if self.terminal_test(GameState('1', 0, board, score)):
        #     # Game is over, collect remaining stones
        #     player1_total = score['1'] + sum(board[i] 
        #                                    for i in range(self.pits_per_player))
        #     player2_total = score['2'] + sum(board[i] 
        #                                    for i in range(self.p1_store + 1, 
        #                                                 self.p2_store))
        #     if player1_total > player2_total:
        #         return 1
        #     elif player2_total > player1_total:
        #         return -1
        #     else:
        #         return 0
        # return 0  # Game is not over
        return board[self.pits_per_player] - board[self.pits_per_player * 2 + 1]



# Creating player classes
below we have classes that defined types of players

In [ ]:
class GameExit(Exception):
    """Custom exception for handling game exits."""
    pass

class HumanPlayer:
    """Interactive player that asks for moves through the command line."""
    
    def __init__(self, name="Human"):
        self.name = name

    def __call__(self, game, state):
        """Make a move by querying standard input."""
        game.display(state)
        valid_moves = game.actions(state)
        
        if not valid_moves:
            return None
            
        prompt = f"\nPlayer {state.current_player}'s turn ({self.name})\n"
        prompt += f"Valid moves are {valid_moves} (or 'exit' to end game): "
        
        while True:
            move = input(prompt).lower().strip()
            if move in ['exit', 'quit', 'escape']:
                raise GameExit("Game ended by player")
            try:
                move = int(move)
                if move in valid_moves:
                    return move
            except ValueError:
                pass
            print('Invalid move. Try again.')

class RandomPlayer:
    """A player that chooses moves randomly from valid options."""
    
    def __init__(self, name="Random"):
        self.name = name

    def __call__(self, game, state):
        """Choose a random valid move."""
        valid_moves = game.actions(state)
        if valid_moves:
            return random.choice(valid_moves)
        return None
    
def play_mancala():
    """Play a game of Mancala with graceful exit handling."""
    try:
        # Create a game instance
        game = Mancala(pits_per_player=3, stones_per_pit=2)
        
        # Create players
        player1 = HumanPlayer("Player 1")
        # player2 = HumanPlayer("Player 2")  # Or use RandomPlayer() for AI opponent
        player2 = RandomPlayer()
        
        # Play the game
        final_state = game.play_game(player1, player2)
        
        # Display final score
        print("\nGame Over!")
        print(f"Final Score - {player1.name}: {final_state.score['1']}, "
              f"{player2.name}: {final_state.score['2']}")
        
        # Determine winner
        if final_state.score['1'] > final_state.score['2']:
            print(f"{player1.name} wins!")
        elif final_state.score['2'] > final_state.score['1']:
            print(f"{player2.name} wins!")
        else:
            print("It's a tie!")
        
    except (KeyboardInterrupt, EOFError):
        print("\nGame terminated by user!")
        sys.exit(0)

if PLAY_NEW_VERSION:
    play_mancala()

# Minimax Player
Now we are going to build an AI player that uses minimax to choose the best move with a variable
number of plies and a utility function we describe to see if it can beat the random player more than 50% of the time.

In [ ]:
def minmax_decision(state, game, depth=5):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), depth - 1))
        return v

    def min_value(state, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), depth - 1))
        return v

    # Body of minmax_decision:
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a), depth - 1))

In [ ]:
class MinimaxPlayer:
    """A player that uses minimax to choose the best move."""
    
    def __init__(self, name="Minimax"):
        self.name = name

    def __call__(self, game, state):
        """Choose a random valid move."""
        move = minmax_decision(state, game)
        return move

In [ ]:
def play_mancala_MM():
    """Play a game of Mancala with graceful exit handling."""
    try:
        # Create a game instance
        game = Mancala(pits_per_player=3, stones_per_pit=2)
        
        # Create players
        player1 = MinimaxPlayer()
        # player2 = HumanPlayer("Player 2")  # Or use RandomPlayer() for AI opponent
        player2 = RandomPlayer()
        
        # Play the game
        final_state = game.play_game(player1, player2)
        
        # Display final score
        print("\nGame Over!")
        print(f"Final Score - {player1.name}: {final_state.score['1']}, "
              f"{player2.name}: {final_state.score['2']}")
        
        # Determine winner
        if final_state.score['1'] > final_state.score['2']:
            print(f"{player1.name} wins!")
            return 1
        elif final_state.score['2'] > final_state.score['1']:
            print(f"{player2.name} wins!")
            return -1
        else:
            print("It's a tie!")
            return 0
        
    except (KeyboardInterrupt, EOFError):
        print("\nGame terminated by user!")
        sys.exit(0)

if PLAY_MINIMAX_VERSION:
    play_mancala()

# Simulating 100 Games(Random player vs Minimax player) using a depth of 5 plies

In [ ]:
minimaxwins = 0
randomwins = 0
ties = 0
for i in range(100):
    result = play_mancala_MM()
    if(result == 1):
        minimaxwins += 1
    elif(result == -1):
        randomwins += 1
    else:
        ties += 1

In [ ]:
print("Percentage of games won by the minimax player:", minimaxwins/100)

# Alpha-Beta Player
Now we are going to build an AI player that uses Alpha-Beta to choose the best move

In [ ]:
def alpha_beta_search(state, game, depth=5):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth - 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth - 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, depth - 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

In [ ]:
class AlphaBetaPlayer:
    """A player that uses Alpha-Beta to choose the best move."""
    
    def __init__(self, name="AlphaBeta"):
        self.name = name

    def __call__(self, game, state):
        """Choose a random valid move."""
        move = alpha_beta_search(state, game)
        return move

In [ ]:
def play_mancala_AB():
    """Play a game of Mancala with graceful exit handling."""
    try:
        # Create a game instance
        game = Mancala(pits_per_player=3, stones_per_pit=2)
        
        # Create players
        player1 = AlphaBetaPlayer()
        # player2 = HumanPlayer("Player 2")  # Or use RandomPlayer() for AI opponent
        player2 = RandomPlayer()
        
        # Play the game
        final_state = game.play_game(player1, player2)
        
        # Display final score
        print("\nGame Over!")
        print(f"Final Score - {player1.name}: {final_state.score['1']}, "
              f"{player2.name}: {final_state.score['2']}")
        
        # Determine winner
        if final_state.score['1'] > final_state.score['2']:
            print(f"{player1.name} wins!")
            return 1
        elif final_state.score['2'] > final_state.score['1']:
            print(f"{player2.name} wins!")
            return -1
        else:
            print("It's a tie!")
            return 0
        
    except (KeyboardInterrupt, EOFError):
        print("\nGame terminated by user!")
        sys.exit(0)

if PLAY_ALPHABETA_VERSION:
    play_mancala()

# Simulating 100 Games(AlphaBeta vs Random) using a depth of 5 plies

In [ ]:
alpha_beta_wins = 0
randomwins = 0
ties = 0
for i in range(100):
    result = play_mancala_AB()
    if(result == 1):
        alpha_beta_wins += 1
    elif(result == -1):
        randomwins += 1
    else:
        ties += 1

In [ ]:
print("Percentage of games won by the minimax player:", alpha_beta_wins/100)

# (Extra Credit)

In [ ]:
def alpha_beta_search(state, game, depth=10):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth - 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth - 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, depth - 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

In [ ]:
alpha_beta_wins = 0
randomwins = 0
ties = 0
for i in range(100):
    result = play_mancala_AB()
    if(result == 1):
        alpha_beta_wins += 1
    elif(result == -1):
        randomwins += 1
    else:
        ties += 1

In [ ]:
print("Percentage of games won by the minimax player:", alpha_beta_wins/100)